##Anjali Kumari

##25/AFI/17

In [ ]:
!pip install torch torchvision torchaudio pandas scikit-learn tqdm -q


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
file_path = "spa.txt"

with open(file_path, "r", encoding="utf-8") as f:
    texts = f.readlines()

texts = [t.strip() for t in texts if t.strip()]

print("Total samples:", len(texts))
print("Sample:", texts[:3])


Total samples: 118964
Sample: ['Go.\tVe.', 'Go.\tVete.', 'Go.\tVaya.']


In [ ]:
train_texts = texts[:80000]
val_texts = texts[80000:90000]
test_texts = texts[90000:100000]

print(len(train_texts), len(val_texts), len(test_texts))


80000 10000 10000


In [ ]:
from collections import Counter

def tokenize(text):
    return text.lower().split()

counter = Counter()

for text in train_texts:
    counter.update(tokenize(text))

vocab = {word: i+2 for i, (word, _) in enumerate(counter.most_common())}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

vocab_size = len(vocab)
print("Vocab size:", vocab_size)


Vocab size: 43556


In [ ]:
MAX_LEN = 20

def encode(text):
    tokens = tokenize(text)
    ids = [vocab.get(tok, 1) for tok in tokens][:MAX_LEN]
    if len(ids) < MAX_LEN:
        ids += [0] * (MAX_LEN - len(ids))
    return ids


In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.data = [encode(t) for t in texts]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx][:-1])
        y = torch.tensor(self.data[idx][1:])
        return x, y


In [ ]:
BATCH_SIZE = 128

train_loader = DataLoader(TextDataset(train_texts), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TextDataset(val_texts), batch_size=BATCH_SIZE)
test_loader = DataLoader(TextDataset(test_texts), batch_size=BATCH_SIZE)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

model = LSTMModel(vocab_size).to(device)
print(model)


LSTMModel(
  (embedding): Embedding(43556, 128)
  (lstm): LSTM(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=43556, bias=True)
)


In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x)

        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


100%|██████████| 625/625 [00:41<00:00, 14.91it/s]


Epoch 1, Loss: 6.9466


100%|██████████| 625/625 [00:43<00:00, 14.36it/s]


Epoch 2, Loss: 5.1664


100%|██████████| 625/625 [00:45<00:00, 13.69it/s]


...

Epoch 18, Loss: 1.5458


100%|██████████| 625/625 [00:45<00:00, 13.82it/s]


Epoch 19, Loss: 1.5072


100%|██████████| 625/625 [00:45<00:00, 13.75it/s]

Epoch 20, Loss: 1.4720


In [ ]:
model.eval()
val_loss = 0

with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        val_loss += loss.item()

print("Validation Loss:", val_loss / len(val_loader))


Validation Loss: 5.503110586842404


In [ ]:
model.eval()
test_loss = 0

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        test_loss += loss.item()

print("Test Loss:", test_loss / len(test_loader))


Test Loss: 6.118708459636833


In [ ]:
def predict_next(text):
    model.eval()
    encoded = torch.tensor([encode(text)[:-1]]).to(device)

    with torch.no_grad():
        output = model(encoded)
        pred = output.argmax(dim=-1)

    inv_vocab = {v: k for k, v in vocab.items()}
    words = [inv_vocab.get(i.item(), "<UNK>") for i in pred[0]]

    return " ".join(words)

print(predict_next("this is a simple"))


is a very of me. fish. two. his. his. his. his. error error error error así así así así
